<a href="https://colab.research.google.com/github/clcinpdx/datascience/blob/master/RAG_Maven_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
#!pip install -r "/content/drive/MyDrive/Colab Notebooks/requirements.txt"
!apt-get -q update
!apt-get -q install -y pandoc
!pip install -q requests
!pip install -q chromadb
!pip install -q braintrust
!pip install -q autoevals
!pip install -q openai
!pip install -q dotenv
!pip install -q langchain_community
!pip install -q langchain_text_splitters
!pip install -q langchain_openai
!pip install -q unstructured
!pip install -q pymupdf
!pip install -q nltk
!pip install -q re
!pip install -q pymupdf
!pip install -q rank_bm25
!pip install -q google-generativeai


print("\nInstallation of libraries complete.\n")


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lis

In [2]:
!pip uninstall -y langchain langchain-core langchain-community langchain-openai
!pip install -q langchain langchain-core langchain-community langchain-openai

Found existing installation: langchain 1.0.2
Uninstalling langchain-1.0.2:
  Successfully uninstalled langchain-1.0.2
Found existing installation: langchain-core 1.0.1
Uninstalling langchain-core-1.0.1:
  Successfully uninstalled langchain-core-1.0.1
Found existing installation: langchain-community 0.4
Uninstalling langchain-community-0.4:
  Successfully uninstalled langchain-community-0.4
Found existing installation: langchain-openai 1.0.1
Uninstalling langchain-openai-1.0.1:
  Successfully uninstalled langchain-openai-1.0.1


In [3]:
from langchain_community.document_loaders import DirectoryLoader
#from langchain_community.document_loaders import UnstructuredMarkdownLoader
#from langchain_community.document_loaders import PyPDFLoader
#from langchain_community.document_loaders import PyMuPDFLoader
from nltk.tokenize import sent_tokenize
import nltk # Import the nltk library
nltk.download('punkt') # Download the punkt tokenizer data
nltk.download('punkt_tab') # Download the punkt_tab tokenizer data

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
from openai import OpenAI
import requests
import json
from google.colab import userdata
import os # Import the os module
import chromadb
import re
import sys
from rank_bm25 import BM25Okapi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
from IPython.display import HTML, display

# Run this once at the top of your notebook
display(HTML("<style>pre { white-space: pre-wrap; }</style>"))

In [5]:
# Load environment variables - project contains .env file with API keys
dotenv_path = '/content/drive/MyDrive/Colab Notebooks/.env'
load_dotenv(dotenv_path)

OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
CHROMADB_API_KEY = userdata.get('CHROMADB_API_KEY')
BRAINTRUST_API_KEY = userdata.get('BRAINTRUST_API_KEY')
os.environ['BRAINTRUST_API_KEY'] = BRAINTRUST_API_KEY
BRAINTRUST_API_URL='https://api.braintrust.dev'

chroma_client = chromadb.CloudClient(api_key=CHROMADB_API_KEY,
  tenant='35ea46f3-d512-4358-b1c4-037c01bc0a79', database='mavin-test')

#set path variables
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/stockdata/NVIDIA"

model_for_qna = "anthropic/claude-sonnet-4.5"
model_for_query = "anthropic/claude-sonnet-4.5"
model_for_eval = "anthropic/claude-sonnet-4.5"
#model_for_eval = "openai/gpt-5"
collection_name = "nvidia"
bm25_collection_name = "bm25-nvidia"
returned_results_semantic = 10
returned_results_bm25 = 5

In [11]:
  # Build the system prompt

base_system_prompt = """
  You are a helpful assistant answering questions about the company NVIDIA.
  But you only answer based on knowledge I'm providing you. You don't use your internal knowledge and you don't make things up.

    IMPORTANT: Provide concise, factual answers. Use specific numbers, dates, and facts directly from the data. Avoid explanations, elaborations, or conversational filler. Answer in as few words as possible.

    Example:
    Question: What was NVIDIA's revenue in fiscal 2025?
    Good answer: $130.5 billion
    Bad answer: NVIDIA had a great year in fiscal 2025, achieving remarkable revenue growth. The company's total revenue reached approximately $130.5 billion, which represents...

--------------------
The data:
"""

In [26]:
import google.generativeai as genai
from google.generativeai import Schema, GenerateContentConfig, Type # <-- Import Schema, GenerateContentConfig, and Type directly from google.generativeai
from langchain_core.documents import Document # Import Document here

def main():

  def print_skynet_logo():
    print("""
    ███████╗██╗  ██╗██╗   ██╗███╗   ██╗███████╗████████╗
    ██╔════╝██║ ██╔╝╚██╗ ██╔╝████╗  ██║██╔════╝╚══██╔══╝
    ███████╗█████╔╝  ╚████╔╝ ██╔██╗ ██║█████╗     ██║
    ╚════██║██╔═██╗   ╚██╔╝  ██║╚██╗██║██╔══╝     ██║
    ███████║██║  ██╗   ██║   ██║ ╚████║███████╗   ██║
    ╚══════╝╚═╝  ╚═╝   ╚═╝  ╚═╝  ╚═══╝╚═══════════╝

    ═══════════════════════════════════════════════════
              CYBERDYNE SYSTEMS - SERIES 800
    ═══════════════════════════════════════════════════
    """)
    return

  def goodbye():
      print("\nThank you for using Skynet!")
      print("\nAll results produced via Skynet Systems @2025, a quality Cyberdyne subsidiary with humanity's best interest always at heart.")

  # Modified function to use Google Gemini 2.0 Flash for PDF processing
  def load_and_process_documents(data_path):
    """Loads and processes PDF documents using Google Gemini 2.0 Flash with guaranteed JSON output."""
    from google.generativeai import GenerativeModel
    # Ensure Document is accessible if not imported globally

    # Define the schema for the expected JSON output: an array of objects, each with a 'text' key.
    # Removed bounding_box from schema as per user request to remove bounding box code
    chunk_schema = Schema(
        type=Type.ARRAY, # Use Type directly after explicit import
        description="A list of document chunks, where each chunk is an object with a 'text' field.",
        items=Schema(
            type=Type.OBJECT, # Use Type directly after explicit import
            properties={
                "text": Schema( # Use Schema for the property definition
                    type=Type.STRING, # Use Type directly after explicit import
                    description="The raw extracted text content of a single document chunk." # Correctly placed description
                )
            },
            required=["text"]
        )
    )

    # Initialize the Gemini API
    try:
        # ... (API initialization remains the same)
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=GOOGLE_API_KEY)
        gemini_model = GenerativeModel('gemini-2.0-flash')
    except Exception as e:
        print(f"Error initializing Google Gemini API: {e}")
        print("Please ensure you have added your GOOGLE_API_KEY to Colab secrets.")
        return []

    all_chunks = []
    for filename in os.listdir(data_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(data_path, filename)
            print(f"Processing {filename}...")
            try:
                # Use Gemini to process the PDF
                with open(file_path, "rb") as f:
                    response = gemini_model.generate_content(
                        contents=[
                            {
                                "role": "user",
                                "parts": [
                                    {"mime_type": "application/pdf", "data": f.read()},
                                    # Modified prompt to be more direct
                                    {"text": "Extract and return the text for each chunk of this PDF document, strictly following the provided JSON schema."},
                                ],
                            }
                        ],
                        # Use the new config parameter with response_schema
                        config=GenerateContentConfig(
                            response_mime_type="application/json",
                            response_schema=chunk_schema
                        )
                    )

                # Attempt to load the JSON response
                pdf_data = None
                try:
                    pdf_data = json.loads(response.text)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from Gemini response for {filename}: {e}")
                    print(f"Raw response text: {response.text}") # Print raw response for debugging
                    continue # Skip to the next file if JSON decoding fails

                # Check if the parsed data is a list and process it
                if isinstance(pdf_data, list):
                    for item in pdf_data:
                        if isinstance(item, dict) and "text" in item: # Check if item is a dictionary and has 'text' key
                            all_chunks.append(Document(
                                page_content=item["text"],
                                metadata={
                                    "source": filename,
                                }
                            ))
                        else:
                            print(f"Skipping item in {filename} due to unexpected structure: {item}")
                else: # Handle cases where pdf_data is not a list
                    print(f"Skipping processing for {filename} due to unexpected JSON structure: Expected a list, but received {type(pdf_data)}")
                    print(f"Received data: {pdf_data}") # Print the received data for debugging

            except Exception as e:
                print(f"Error processing {filename} with Gemini: {e}")

    print(f"Processed {len(all_chunks)} chunks with metadata.")
    return all_chunks


  # Removed sentence_aware_chunking as Gemini handles chunking
  # def sentence_aware_chunking(documents, max_len):
  #   all_chunks = []
  #   for doc in documents:
  #       sentences = sent_tokenize(doc.page_content)
  #       current_chunk = []
  #       current_len = 0
  #       for sentence in sentences:
  #           # Check if adding the current sentence exceeds the max length
  #           if current_len + len(sentence) + (1 if current_chunk else 0) > max_len:
  #               # If it does, add the current accumulated chunk to all_chunks
  #               if current_chunk:
  #                   all_chunks.append(Document(page_content=' '.join(current_chunk), metadata=doc.metadata))
  #               # Start a new chunk with the current sentence
  #               current_chunk = [sentence]
  #               current_len = len(sentence)
  #           else:
  #               # Otherwise, add the current sentence to the current chunk
  #               current_chunk.append(sentence)
  #               current_len += len(sentence) + (1 if current_chunk else 0)  # Add space if not the first sentence

  #       # Add the last chunk if it's not empty
  #       if current_chunk:
  #             all_chunks.append(Document(page_content=' '.join(current_chunk), metadata=doc.metadata))
  #   return all_chunks

  def save_to_chroma(chunks):
    collection = chroma_client.get_or_create_collection(name=collection_name)
    batch_size = 300
    for i in range(0, len(chunks), batch_size):
      batch = chunks[i:i + batch_size]
      documents = []
      metadata = []
      ids = []
      for j, chunk in enumerate(batch):
          documents.append(chunk.page_content)
          ids.append("ID" + str(i + j)) # Ensure unique IDs across all batches
          metadata.append(chunk.metadata)

      print(f"Attempting to add batch {int(i/batch_size) + 1} of {int(len(chunks)/batch_size) + 1} with {len(batch)} chunks.")
      try:
          collection.upsert(documents=documents, metadatas=metadata, ids=ids)
          print(f"Successfully added batch {int(i/batch_size) + 1}.")
      except Exception as e:
          print(f"Error adding batch {int(i/batch_size) + 1}: {e}")
          # Depending on the error, you might want to break or handle it differently

    return collection

  # Create Keyword Tokenized Corpus
  def create_keyword_corpus(chunks): # Modified to accept chunks as an argument
    # Extract text content from chunks
    corpus = [chunk.page_content for chunk in chunks]
    # Tokenize each document in the corpus
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    # Store the tokenized corpus in a variable
    # This is already done by assigning to tokenized_corpus
    print(f"Prepared a tokenized corpus of {len(tokenized_corpus)} documents for BM25.")
    # Create a BM25 model instance
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25, tokenized_corpus


  # Save BM25 data
  def save_bm25_data(bm25_model, tokenized_corpus, collection_name):
      """Saves the BM25 model and tokenized corpus to disk."""
      bm25_dir = f"./bm25_data/{collection_name}"
      os.makedirs(bm25_dir, exist_ok=True)
      # Save the tokenized corpus
      with open(f"{bm25_dir}/tokenized_corpus.json", "w") as f:
        json.dump(tokenized_corpus, f)
      # BM25Okapi model itself doesn't have a simple save/load method.
      # Rebuilding from the tokenized corpus is the standard approach.
      print(f"Tokenized corpus saved to {bm25_dir}/tokenized_corpus.json")


  # Load BM25 data
  def load_bm25_data(collection_name):
      """Loads the tokenized corpus and rebuilds the BM25 model from disk."""
      bm25_dir = f"./bm25_data/{collection_name}"
      corpus_path = f"{bm25_dir}/tokenized_corpus.json"
      if not os.path.exists(corpus_path):
        print(f"BM25 data not found at {corpus_path}")
        return None, None
      with open(corpus_path, "r") as f:
        tokenized_corpus = json.load(f)
      bm25_model = BM25Okapi(tokenized_corpus)
      print(f"BM25 model loaded from {corpus_path}")
      return bm25_model, tokenized_corpus

  def rerank_results(chroma_results, bm25_results, query):
    """Combines and reranks results from ChromaDB and BM25."""
    combined_docs = []
    seen_content = set()

    print("\n" + "="*80)
    print("COMBINED SEARCH RESULTS")
    print("="*80)

    # Add ChromaDB results
    chroma_count = 0
    if chroma_results and chroma_results.get('documents'):
        print("\n📊 SEMANTIC SEARCH (ChromaDB) RESULTS:")
        print("-"*80)
        for doc_list in chroma_results['documents']:
            for doc_content in doc_list:
                if doc_content not in seen_content:
                    chroma_count += 1
                    combined_docs.append(doc_content)
                    seen_content.add(doc_content)
                    print(f"\n[Semantic #{chroma_count}]")
                    print(f"{doc_content}")
                    print("-"*80)

    # Add BM25 results (AKA Keyword search)
    bm25_count = 0
    if bm25_results:
        print("\n🔍 KEYWORD SEARCH (BM25) RESULTS:")
        print("-"*80)
        for doc_content in bm25_results:
            # BM25 results should already be strings from your corpus
            # If they're tokenized, join them
            if isinstance(doc_content, list):
                doc_content = " ".join(doc_content)

            if doc_content not in seen_content:
                bm25_count += 1
                combined_docs.append(doc_content)
                seen_content.add(doc_content)
                print(f"\n[Keyword #{bm25_count}]")
                print(f"{doc_content}")
                print("-"*80)

    # Summary
    print("\n" + "="*80)
    print(f"SUMMARY: {chroma_count} semantic + {bm25_count} keyword = {len(combined_docs)} total unique documents")
    print("="*80 + "\n")

    return combined_docs


  # Define the function to perform BM25 search
  def perform_bm25_search(bm25, tokenized_corpus, query_string,returned_results_bm25): # Added bm25 model as argument
    """Performs a BM25 search on the tokenized corpus
    Args:
    bm25: The BM25 model instance.
    tokenized_corpus: The tokenized corpus used to build the BM25 model.
    query_string: The query string to search for.
    n: The number of top results to return.

    Returns:
    A list of top n documents from the corpus based on the BM25 score.
    """
    tokenized_query = query_string.split(" ")
    top_docs = bm25.get_top_n(tokenized_query, tokenized_corpus, returned_results_bm25) # Added tokenized_corpus
    return top_docs



  #################################
  # Evaluation -> Braintrust (bt) #
  #################################

  def get_braintrust_results_task(braintrust_input):

      collection = chroma_client.get_collection(name=collection_name)
      bm25_model, tokenized_corpus = load_bm25_data(bm25_collection_name)
      if bm25_model is None or tokenized_corpus is None:
        print("BM25 data not found. Please build the Keyword DB first ('k' option).")

      # Perform Semantic (vector) Search
      semantic_results = collection.query(
      query_texts=[braintrust_input],
      n_results=returned_results_semantic
      )

      # Perform BM25 search
      bm25_results = perform_bm25_search(bm25_model, tokenized_corpus, braintrust_input, returned_results_bm25) # Pass bm25_model and tokenized_corpus

      # Rerank the combined results
      combined_context_docs = rerank_results(semantic_results, bm25_results, braintrust_input)

      # Build the system prompt using the combined context
      context_string = "\n".join(combined_context_docs)
      system_prompt = f"{base_system_prompt}{context_string}"

      print(f"braintrust_input: {braintrust_input}")


      response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "HTTP-Referer": "https://colab.research.google.com/",
        "X-Title": "Colab Notebook",
        },
        data=json.dumps({
        "model": model_for_query,
        "messages": [
            {"role":"system","content":system_prompt},
            {"role":"user","content":braintrust_input}
          ]
        })
        )

      output = response.json()['choices'][0]['message']['content']
      return output


  #############################
  # Evalulation -> Braintrust #
  #############################

  def eval_model():
      from braintrust import Eval, init_dataset
      from autoevals import (Factuality, Faithfulness, ContextRelevancy, AnswerRelevancy, Levenshtein)

      eval_run = Eval(
        "mavin",
        data=init_dataset(project="mavin", name="Synthetic-Eval-Q_n_Answers-Sonnet45"),
        task=get_braintrust_results_task,
        scores=[Factuality],
        #scores=[Factuality,Faithfulness,ContextRelevancy,AnswerRelevancy],
        experiment_name=f"Eval-{collection_name}-LLM_used-{model_for_eval}__keyword_result_count-{returned_results_bm25}__vector_result_count-{returned_results_semantic}",
        max_concurrency=5
        )
      return


  #############################
  # Adhoc query of the system #
  #############################

  def adhoc_ask(collection, bm25_model, tokenized_corpus): # Added bm25_model and tokenized_corpus as arguments

    user_query = input(f"What do you want to know about {collection_name}?\n\n")

    # Perform Semantic (vector) Search
    results = collection.query(
    query_texts=[user_query],
    n_results=returned_results_semantic,
    include=['metadatas'] # Include metadata
    )

    # Perform BM25 search
    bm25_results = perform_bm25_search(bm25_model, tokenized_corpus, user_query, returned_results_bm25) # Pass bm25_model and tokenized_corpus
    #print(f"BM25 Results:\n")
    #for doc_tokens in bm25_results:
    #  print(" ".join(doc_tokens))
    #print("\n")

    # Rerank the combined results
    combined_context_docs = rerank_results(results, bm25_results, user_query) # Pass user_query as query

    # Build the system prompt using the combined context
    context_string = "\n".join(combined_context_docs)
    system_prompt = f"{base_system_prompt}{context_string}"

    print(system_prompt)

    response = requests.post(
      url="https://openrouter.ai/api/v1/chat/completions",
      headers={
      "Authorization": f"Bearer {OPENROUTER_API_KEY}",
      "HTTP-Referer": "https://colab.research.google.com/",
      "X-Title": "Colab Notebook",
      },
      data=json.dumps({
      "model": model_for_query,
      "messages": [
          {"role":"system","content":system_prompt},
          {"role":"user","content":user_query}
        ]
      })
      )

    answer = response.json()['choices'][0]['message']['content']
    print(f"{answer}")

    # Removed code to extract and display bounding box information


    return answer

  print_skynet_logo()

  while True:
    user_input = input(
        "\nPlease select an option:\n"
        "  'b' - Build VectorDB RAG\n"
        "  'k' - Build Keyword (BM25) DB\n"
        "  'e' - Run evals\n"
        "  'q' - Query the RAG directly\n"
        "  'x' - Exit\n"
        "Your choice: "
    ).strip().lower()

    if user_input == 'b':
        chunks = load_and_process_documents(DATA_PATH) # Use the new function
        collection = save_to_chroma(chunks)
        print(f"Successfully built VectorDB RAG: {collection}")

    elif user_input == 'k': # Added option for building keyword DB
        chunks = load_and_process_documents(DATA_PATH) # Use the new function to get chunks for BM25
        bm25_model, tokenized_corpus = create_keyword_corpus(chunks)
        save_bm25_data(bm25_model, tokenized_corpus, bm25_collection_name)
        print(f"Successfully built and saved Keyword (BM25) DB for collection: {bm25_collection_name}")

    elif user_input == 'e':
        eval_model()
        print("\nSuccessfully initiated eval -> see results in Braintrust.")
        goodbye()
        break



    elif user_input == 'q':
        while True:
            collection = chroma_client.get_collection(name=collection_name)
            bm25_model, tokenized_corpus = load_bm25_data(bm25_collection_name) # Load BM25 data
            if bm25_model is None or tokenized_corpus is None:
              print("BM25 data not found. Please build the Keyword DB first ('k' option).")
              break # Exit query loop if BM25 data is not available

            adhoc_ask(collection, bm25_model, tokenized_corpus) # Pass BM25 data to ask function
            query_input = input("\nPress 'Enter' to continue asking questions, or 'x' to exit: ").strip().lower()
            if query_input == 'x':
              goodbye()
              break

    elif user_input == 'x':
      goodbye()
      break

    else:
        print("Invalid input. Please choose 'b', 'k', 'e', 'q', or 'x'.") # Updated valid options





if __name__ == "__main__":
    main()

ImportError: cannot import name 'Schema' from 'google.generativeai' (/usr/local/lib/python3.12/dist-packages/google/generativeai/__init__.py)